In [36]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score,GridSearchCV

In [93]:
#데이터 불러오기 cut_id를 인덱스로
X_train = pd.read_csv("X_train.csv", encoding='euc-kr', index_col='cust_id')
y_train = pd.read_csv("y_train.csv", encoding='euc-kr', index_col='cust_id')

X_test = pd.read_csv("X_test.csv", encoding='euc-kr', index_col='cust_id')

In [94]:
X_train.shape, y_train.shape, X_test.shape

((3500, 9), (3500, 1), (2482, 9))

In [95]:
X_train.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
cust_id,,,,,,,,,
0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [96]:
y_train.head()

,gender
cust_id,
0,0
1,0
2,1
3,1
4,0


In [97]:
X_train.info()

#환불금액 null값 있음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3500 entries, 0 to 3499
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   총구매액     3500 non-null   int64  
 1   최대구매액    3500 non-null   int64  
 2   환불금액     1205 non-null   float64
 3   주구매상품    3500 non-null   object 
 4   주구매지점    3500 non-null   object 
 5   내점일수     3500 non-null   int64  
 6   내점당구매건수  3500 non-null   float64
 7   주말방문비율   3500 non-null   float64
 8   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 273.4+ KB


#### 결측치 처리
- 환불금액의 경우 결측치가 절반 이상이 되어 향후 데이터 분석 시 제외하고 진행하면 됨
- 나머지는 결측치 없음

#### 범주형 데이터 원핫인코딩
- 주구매상품과 주구매지점은 범주형 데이터이므로 원핫인코딩

In [98]:
X_train['주구매상품'].nunique()

42

In [99]:
X_test['주구매상품'].nunique()

41

- 주구매 상품의 경우 train 데이터에는 있는 데이터가 test에 없는 경우가 잇어서 train과 test 데이터를 병합한 후 원핫인코딩 하고 다시 나눠주는 형태로 진행

In [100]:
X_train['주구매상품']

cust_id
0           기타
1          스포츠
2       남성 캐주얼
3           기타
4           보석
         ...  
3495        골프
3496      시티웨어
3497      주방용품
3498       화장품
3499        기타
Name: 주구매상품, Length: 3500, dtype: object

In [101]:
X_test['주구매상품']

cust_id
3500        골프
3501       농산물
3502      가공식품
3503      주방용품
3504       수산품
         ...  
5977        골프
5978      섬유잡화
5979    남성 캐주얼
5980        육류
5981      가공식품
Name: 주구매상품, Length: 2482, dtype: object

In [102]:
#주구매상품만 다르므로 주구매상품만 진행
temp1 = pd.concat([X_train['주구매상품'],X_test['주구매상품']],axis=0)

In [103]:
temp1

cust_id
0           기타
1          스포츠
2       남성 캐주얼
3           기타
4           보석
         ...  
5977        골프
5978      섬유잡화
5979    남성 캐주얼
5980        육류
5981      가공식품
Name: 주구매상품, Length: 5982, dtype: object

In [104]:
temp1_encoding = pd.get_dummies(temp1)
X_train_encoding = temp1_encoding.iloc[:3500]
X_test_encoding = temp1_encoding.iloc[3500:]

In [105]:
temp1_encoding

,가공식품,가구,건강식품,골프,구두,기타,남성 캐주얼,남성 트랜디,남성정장,농산물,...,주방용품,차/커피,축산가공,침구/수예,캐주얼,커리어,통신/컴퓨터,트래디셔널,피혁잡화,화장품
cust_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5979,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
X_train_encoding.shape,X_test_encoding.shape

((3500, 42), (2482, 42))

In [68]:
len(X_train_encoding.columns)

42

In [69]:
len(X_test_encoding.columns)

42

In [107]:
#원본 데이터에 원핫 인코딩 한 결과값을 병합한 후 기존 주구매상품 삭제
X_train = pd.concat([X_train,X_train_encoding ],axis=1)
X_train.drop(['주구매상품'], axis=1, inplace=True)


X_train_dummies = pd.get_dummies(X_train['주구매지점'])
X_train = pd.concat([X_train,X_train_dummies],axis=1)
X_train.drop(['주구매지점'], axis=1, inplace=True)

### 모델링
- 환불금액 제외(결측치 많음0
- 범주형 데이터 일단 제외(주구매상품, 주구매지점)

In [108]:
#분석 시 안쓰는 칼럼 삭제
X_train.drop('환불금액', axis=1, inplace=True)
X_test.drop('환불금액', axis=1, inplace=True)

In [86]:
X_train.columns.to_frame()

,0
총구매액,총구매액
최대구매액,최대구매액
내점일수,내점일수
내점당구매건수,내점당구매건수
주말방문비율,주말방문비율
...,...
커리어,커리어
통신/컴퓨터,통신/컴퓨터
트래디셔널,트래디셔널
피혁잡화,피혁잡화


In [109]:
#xgboost 모델
model = XGBClassifier()
result = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
result.mean()

[13:05:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:05:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:05:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:05:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

0.6263151745225134

In [110]:
# 로지스틱 회귀모델
model = LogisticRegression()
result = cross_val_score(model, X_train,y_train,cv=5, scoring='roc_auc')
result.mean()

0.6262544376778972

In [111]:
#원본 데이터에 원핫 인코딩 한 결과값을 병합한 후 기존 주구매상품 삭제
X_test = pd.concat([X_test,X_test_encoding ],axis=1)
X_test.drop(['주구매상품'], axis=1, inplace=True)


X_test_dummies = pd.get_dummies(X_test['주구매지점'])
X_test = pd.concat([X_test,X_test_dummies],axis=1)
X_test.drop(['주구매지점'], axis=1, inplace=True)

In [112]:
#최적의 파라미터로 분석
model = RandomForestClassifier(random_state=3, max_depth=7, max_features=0.2, min_samples_leaf=10, n_estimators=50)
model.fit(X_train, y_train)

#예측
pre = model.predict(X_test)

In [115]:
pre

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## 제출 결과물

In [116]:
dic = {'cust_id':X_test.index, 'gender':pre}
submit = pd.DataFrame(dic)
submit

,cust_id,gender
0,3500,0
1,3501,0
2,3502,0
3,3503,0
4,3504,0
...,...,...
2477,5977,0
2478,5978,1
2479,5979,0
2480,5980,0
